In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import NMF
from scipy import sparse

In [2]:
ratings_data = pd.read_csv("training.csv")
ratings_data.head()

,user,movie,rating,timestamp
0,6040,858,4,956703932
1,6040,593,5,956703954
2,6040,2384,4,956703954
3,6040,1961,4,956703977
4,6040,2019,5,956703977


In [3]:
movie_data = pd.read_csv("movies.dat",delimiter = "::",names=["movie","title","genre"])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
movie_data.head()

,movie,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
dummy_cols = movie_data.genre.str.get_dummies()

In [6]:
movie_data = pd.concat((movie_data,dummy_cols),axis = 1)

In [7]:
movie_data= movie_data.drop("genre",axis=1)

In [8]:
user_data = pd.read_csv("users.dat",delimiter = "::",names=["user","gender","age","occupation","zipcode"])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [9]:
user_data.head()

,user,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [10]:
movie_data["year"]=movie_data["title"]
movie_data["year"] = movie_data["year"].apply(lambda x: x[-5:-1])
movie_data.head()

,movie,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji (1995),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1995
3,4,Waiting to Exhale (1995),0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1995
4,5,Father of the Bride Part II (1995),0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995


In [11]:
movie_data["title"] = movie_data["title"].apply(lambda x: x[:-7])
movie_data.head()

,movie,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1995
3,4,Waiting to Exhale,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1995
4,5,Father of the Bride Part II,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995


In [13]:
movie_data.head()

,movie,title,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1995
3,4,Waiting to Exhale,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1995
4,5,Father of the Bride Part II,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1995


In [14]:
user_data["gender"] = user_data["gender"].map({"M":1,"F":0})

In [15]:
user_data.head()

,user,gender,age,occupation,zipcode
0,1,0,1,10,48067
1,2,1,56,16,70072
2,3,1,25,15,55117
3,4,1,45,7,02460
4,5,1,25,20,55455


In [16]:
movie_rating = pd.merge(ratings_data,movie_data,how="left",left_on ="movie",right_on="movie")

In [17]:
movie_rating.head(10)

,user,movie,rating,timestamp,title,Action,Adventure,Animation,Children's,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,6040,858,4,956703932,"Godfather, The",1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1972
1,6040,593,5,956703954,"Silence of the Lambs, The",0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1991
2,6040,2384,4,956703954,Babe: Pig in the City,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,1998
3,6040,1961,4,956703977,Rain Man,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1988
4,6040,2019,5,956703977,Seven Samurai (The Magnificent Seven) (Shichin...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1954
5,6040,1419,3,956704056,Walkabout,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1971
6,6040,573,4,956704056,"Ciao, Professore! (Io speriamo che me la cavo )",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1993
7,6040,3111,5,956704056,Places in the Heart,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1984
8,6040,213,5,956704056,Burnt By the Sun (Utomlyonnye solntsem),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1994
9,6040,3505,4,956704056,No Way Out,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1987


In [18]:
user_rating = pd.merge(ratings_data,user_data,how="left",left_on ="user",right_on="user")

In [19]:
user_rating.sample(10)

,user,movie,rating,timestamp,gender,age,occupation,zipcode
366946,3572,3468,5,966735767,1,35,11,04287
133932,5076,3552,4,962421573,1,25,3,19147
248992,5875,1,5,965286616,1,25,4,19103
762827,1137,2719,3,975362963,1,18,4,13165
438894,3142,1247,5,968989388,1,25,20,85719
736138,980,1136,5,975105755,1,25,6,92014
758654,859,3753,4,975344465,1,25,12,10128
779698,770,1495,1,975492264,1,18,4,98499
744335,1928,1562,3,975189334,1,1,10,94501
431198,3191,1274,4,968652274,1,25,12,27707


In [20]:
final_train = pd.merge(movie_rating,user_rating,on=["user","movie","rating","timestamp"])

In [21]:
final_train.head()

,user,movie,rating,timestamp,title,Action,Adventure,Animation,Children's,Comedy,...,Romance,Sci-Fi,Thriller,War,Western,year,gender,age,occupation,zipcode
0,6040,858,4,956703932,"Godfather, The",1,0,0,0,0,...,0,0,0,0,0,1972,1,25,6,11106
1,6040,593,5,956703954,"Silence of the Lambs, The",0,0,0,0,0,...,0,0,1,0,0,1991,1,25,6,11106
2,6040,2384,4,956703954,Babe: Pig in the City,0,0,0,1,1,...,0,0,0,0,0,1998,1,25,6,11106
3,6040,1961,4,956703977,Rain Man,0,0,0,0,0,...,0,0,0,0,0,1988,1,25,6,11106
4,6040,2019,5,956703977,Seven Samurai (The Magnificent Seven) (Shichin...,1,0,0,0,0,...,0,0,0,0,0,1954,1,25,6,11106


In [22]:
def get_ratings_data(df):
    '''
    Returns:
        ratings_contents (dataframe): has columns "user", "movie", "rating"
        ratings_as_mat (sparse matrix): rows correspond to users and columns correspond
        to movies. Each element is the user's rating for that movie.
    '''
    ratings_contents = df[['user','movie','rating']]
    highest_user_id = ratings_contents.user.max()
    highest_movie_id = ratings_contents.movie.max()
    ratings_as_mat = sparse.lil_matrix((highest_user_id, highest_movie_id))
    for _, row in ratings_contents.iterrows():
        # subtract 1 from id's due to 0 indexing
        ratings_as_mat[row.user - 1, row.movie - 1] = row.rating
    return ratings_contents, ratings_as_mat

In [25]:
contents, ratings_mat = get_ratings_data(final_train)

In [82]:
ratings_arr = ratings_mat.toarray()
ratings_arr[ratings_arr == 0]=3
ratings_arr.shape

(6040, 3952)

In [75]:
pd_ratings = pd.DataFrame(ratings_arr)

In [78]:
pd_ratings.describe()

,0,1,2,3,4,5,6,7,8,9,...,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951
count,6040.000000,6040.000000,6040.00000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,...,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000,6040.000000
mean,3.331623,3.018543,3.00149,2.993377,2.999834,3.113576,3.025331,2.999669,2.995695,3.062086,...,2.997682,2.999172,2.992715,2.994536,3.001821,3.053642,3.016722,3.000331,3.003146,3.024338
std,0.695211,0.305039,0.27206,0.155346,0.200595,0.445412,0.263444,0.090991,0.127316,0.349986,...,0.101298,0.053050,0.121861,0.117107,0.100487,0.335714,0.175173,0.060356,0.084324,0.219674
min,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
25%,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
max,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [29]:
ratings_mat.nonzero()

(array([ 635,  635,  635, ..., 6039, 6039, 6039], dtype=int32),
 array([ 109,  195,  259, ..., 3734, 3750, 3818], dtype=int32))

In [31]:
contents.shape

(800000, 3)

In [37]:
y =contents["rating"]
X=contents

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25)

In [39]:
_,x_train_um = get_ratings_data(X_train)

In [79]:
x_train_um.shape

(6040, 3952)

In [80]:
x_test_um.shape

(6040, 3952)

In [81]:
contents.shape

(800000, 3)

In [41]:
skmodel = NMF(n_components=5, init='random', random_state=0,alpha = 0.1, l1_ratio = 0.5)

In [42]:
recommender2 = skmodel.fit(x_train_um)

In [43]:
_,x_test_um = get_ratings_data(X_test)

In [44]:
y_pred = recommender2.transform(x_test_um)

In [47]:
H = recommender2.components_

In [48]:
H.shape

(5, 3952)

In [46]:
y_pred.shape

(6040, 5)

In [50]:
pred_final = y_pred.dot(H)

In [51]:
pred_final.shape

(6040, 3952)

In [60]:
pred_final

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.25066702e-02, 1.54589731e-02, 1.09353717e-02, ...,
        2.96399225e-04, 1.61708629e-04, 3.22932701e-03],
       [2.88555674e-01, 7.78842784e-02, 5.85432026e-02, ...,
        2.05149611e-03, 0.00000000e+00, 1.12853163e-02],
       [4.38743463e-01, 5.30726310e-02, 3.09786070e-02, ...,
        4.30787720e-03, 6.09475331e-03, 6.14880416e-02]])

In [98]:
pd_y_pred = pd_y_pred.fillna(3.2)

In [99]:
pd_y_pred.rating == 3.2

,user,movie,rating,prediction
count,159972.000000,159972.000000,159972.000000,159972.000000
mean,3408.062473,1850.770822,3.592904,3.431607
std,1546.836243,1088.980332,1.119684,0.669331
min,636.000000,1.000000,1.000000,0.272625
25%,2039.000000,1023.000000,3.000000,3.032485
50%,3509.000000,1798.000000,4.000000,3.499867
75%,4705.000000,2759.000000,4.000000,3.906292
max,6040.000000,3952.000000,5.000000,5.472911
